# Chapter 10. Proccesing text
## Notebook for Python

### 10.1 Reading and cleaning text

In [16]:
import string

my_wiki_text = "<p><b>Communication</b> (from Latin <i>communicare</i>, meaning to share)"
print(my_wiki_text)
my_wiki_text_2 = my_wiki_text.replace('<p>',' ').replace('<b>',' ').replace('</b>',' ').replace('<i>',' ').replace('</i>',' ') #remove HTML tags
my_wiki_text_2 = "".join([l for l in my_wiki_text_2 if l not in string.punctuation]) #Remove punctuation
my_wiki_text_2 = my_wiki_text_2.lower()  #Convert to lower case
my_wiki_text_2 = " ".join(my_wiki_text_2.split()) #Remove double spaces 
print(my_wiki_text_2)

<p><b>Communication</b> (from Latin <i>communicare</i>, meaning to share)
communication from latin communicare meaning to share


In [17]:
from bs4 import BeautifulSoup 
my_wiki_text_3 = BeautifulSoup(my_wiki_text, 'html.parser')
my_wiki_text_3.get_text()

'Communication (from Latin communicare, meaning to share)'

In [18]:
import re
my_wiki_text_4 = re.sub('<.*?>', "", my_wiki_text)
print(my_wiki_text_4)

Communication (from Latin communicare, meaning to share)


In [19]:
tweets = "<@born_in_america: My second favorite color is green, I must acknowlege my friends for that!>, <@born_in_britain: My second favourite colour is red, I must aknowledge my friends for that!>"
print(tweets)
tweets_unified = re.sub("acknowlege|aknowledge" , "acknowledge", tweets)
tweets_unified = re.sub("col.+?r", "color", tweets_unified)
tweets_unified = re.sub("fav.+?rite" , "favorite", tweets_unified)
print(tweets_unified)

<@born_in_america: My second favorite color is green, I must acknowlege my friends for that!>, <@born_in_britain: My second favourite colour is red, I must aknowledge my friends for that!>
<@born_in_america: My second favorite color is green, I must acknowledge my friends for that!>, <@born_in_britain: My second favorite color is red, I must acknowledge my friends for that!>


In [20]:
users = re.compile(r'\B@[\w_]+').findall(tweets)
print(users)

['@born_in_america', '@born_in_britain']


In [21]:
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
#tweets = "".join([l for l in tweets if l not in string.punctuation])
tokens = word_tokenize(tweets, language='english')
print(tokens)
#print(type(word_list_en))
print(len(tokens))

stop_words_en = set(stopwords.words("english"))
filtered_tokens = [w for w in tokens if not w in stop_words_en]
print(filtered_tokens)
len(filtered_tokens)

['<', '@', 'born_in_america', ':', 'My', 'second', 'favorite', 'color', 'is', 'green', ',', 'I', 'must', 'acknowlege', 'my', 'friends', 'for', 'that', '!', '>', ',', '<', '@', 'born_in_britain', ':', 'My', 'second', 'favourite', 'colour', 'is', 'red', ',', 'I', 'must', 'aknowledge', 'my', 'friends', 'for', 'that', '!', '>']
41
['<', '@', 'born_in_america', ':', 'My', 'second', 'favorite', 'color', 'green', ',', 'I', 'must', 'acknowlege', 'friends', '!', '>', ',', '<', '@', 'born_in_britain', ':', 'My', 'second', 'favourite', 'colour', 'red', ',', 'I', 'must', 'aknowledge', 'friends', '!', '>']


33

In [94]:
from nltk.stem import SnowballStemmer
ss_en = SnowballStemmer('english')
print(ss_en.stem('buildings'))
print(ss_en.stem('builds'))
print(ss_en.stem('awesome'))

build
build
awesom


In [139]:
import spacy
#!python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp(tweets)
print("Number of tokens: ", len(doc))

Number of tokens:  39


In [143]:
print(doc[7].text, doc[7].pos_, doc[7].tag_, doc[7].lemma_)
print(doc[14].text, doc[14].pos_, doc[14].tag_, doc[14].lemma_)

#For the whole text:
#for token in doc:
    #print(token.text, token.lemma_, token.pos_, token.tag_)'''

is AUX VBZ be
friends NOUN NNS friend


In [149]:
noun_chunks = list(doc.noun_chunks)
print("Number of noun chunks: ", len(noun_chunks))
print("Noun chunk 4: ", noun_chunks[3].text) 

#For the whole text:
#for chunk in doc.noun_chunks:
    #print(chunk.text, chunk.root.dep_)

Number of noun chunks:  7
Nuoun chunk 4:  my friends


In [191]:
import nltk
from nltk.parse.corenlp import CoreNLPParser
parser = CoreNLPParser(url='http://localhost:9000')
parse , = parser.raw_parse('My second favorite color is green') 
parse.pretty_print()            

              ROOT                      
               |                         
               S                        
        _______|________________         
       |                        VP      
       |                     ___|____    
       NP                   |       ADJP
  _____|_______________     |        |   
PRP$   JJ      JJ      NN  VBZ       JJ 
 |     |       |       |    |        |   
 My  second favorite color  is     green



In [202]:
docp = nlp ("My second favorite color is green")
for token in docp:
    print(token.text, token.dep_)

My poss
second amod
favorite amod
color nsubj
is ROOT
green acomp


In [198]:
displacy.render(docp, style='dep', jupyter=True, options={'distance': 90})

In [214]:
headline= nlp("Madrid will host Olympic Games in 2032, Pedro Sanchez announced")
entities=[(i, i.label_, i.label) for i in headline.ents]
entities

[(Madrid, 'GPE', 384),
 (Olympic Games, 'ORG', 383),
 (2032, 'DATE', 391),
 (Pedro Sanchez, 'PERSON', 380)]

In [215]:
displacy.render(headline, style = "ent",jupyter = True)